In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/davis-and-kiba/kiba.txt
/kaggle/input/davis-and-kiba/davis.txt
/kaggle/input/davis-and-kiba/davis-filter.txt


# **Processing the Data and encoding sequence for MLP** 

In [4]:
davis_filter_dir = '/kaggle/input/davis-and-kiba/davis-filter.txt'
kiba_dir = '/kaggle/input/davis-and-kiba/kiba.txt'
davis_dir = '/kaggle/input/davis-and-kiba/davis.txt'

In [5]:
col_name = ['drug_id', 'prot_id', 'SMILES', 'sequence amino acid', 'sequence amino acid affinity']
dd = pd.read_csv(davis_dir, sep=' ', header=None)  
dd.columns = col_name
dd = pd.DataFrame(dd)

In [6]:
ddf = pd.read_csv(davis_filter_dir, sep=' ', header=None)  # TODO modify the dataset_dir
ddf.columns = col_name
ddf = pd.DataFrame(ddf)

In [7]:
dk = pd.read_csv(kiba_dir, sep=' ', header=None)  # TODO modify the dataset_dir
dk.columns = col_name
dk = pd.DataFrame(dk)

In [8]:
print("Davis_shape:", dd.shape)
print("Davis_filter_shape:", ddf.shape)
print("kiba_shape:", dk.shape)

Davis_shape: (30056, 5)
Davis_filter_shape: (9125, 5)
kiba_shape: (118254, 5)


In [9]:
amino_acids = "ARNDCEQGHILKMFPSTWYV"  # 20 standard amino acids
amino_acid_to_number = {aa: idx + 1 for idx, aa in enumerate(amino_acids)}
def encode_sequence(sequence):
    return [amino_acid_to_number[aa] for aa in sequence if aa in amino_acid_to_number]

In [10]:
dd['encoded_sequence'] = dd['sequence amino acid'].apply(encode_sequence)
ddf['encoded_sequence'] = ddf['sequence amino acid'].apply(encode_sequence)
dk['encoded_sequence'] = dk['sequence amino acid'].apply(encode_sequence)

# **Evaluation Metrics**

In [23]:
def h(m):
    if m > 0:
        return 1
    elif m == 0:
        return 0.5
    else:
        return 0

def ConcordanceIndex(Affinities,Predictions):
    Z = 0 
    concordant_pairs = 0
    
    n = len(Affinities)
    for i in range(n):
        for j in range(i + 1, n):
            if Affinities[i] > Affinities[j]:
                Z += 1
                concordant_pairs += h(Predictions[i] - Predictions[j])
    
    if Z > 0:
        return concordant_pairs / Z
    else: 
        return 0

In [12]:
def mean_squared_error(Affinities,Predictions):
    errors = np.array(Affinities) - np.array(Predictions)
    return np.mean(errors**2)

In [13]:
from sklearn.linear_model import LinearRegression

def R2(Affinities, Predictions):
    Affinities = np.array(Affinities)
    Predictions = np.array(Predictions)
    
    SST = np.sum((Affinities - np.mean(Affinities))**2)
    SSR = np.sum((Affinities - Predictions)**2)
    
    return 1 - (SSR/SST)

def R02(Affinities, Predictions):
    Affinities = np.array(Affinities).reshape(-1, 1)
    Predictions = np.array(Predictions).reshape(-1, 1)
    
    model = LinearRegression(fit_intercept=False)
    model.fit(Predictions, Affinities)
    Predictions = model.predict(Predictions)
    
    SST = np.sum((Affinities - np.mean(true_values))**2)
    SSR = np.sum((Affinities - predictions)**2)
    
    return 1 - (SSR/SST)

def RMSsquared(Affinities, Predictions):
    Rsquared = R2(Affinities, Predictions)
    R0squared = R02(Affinities, Predictions)
    
    return Rsquared * (1 - np.sqrt(Rsquared - R0squared))

In [14]:
def Pearson_correlation(Affinities, Predictions):
    Affinities = np.array(Affinities)
    Predictions = np.array(Predictions)
    
    TrueAvg = np.mean(Affinities)
    PredAvg = np.mean(Predictions)
    
    Num = np.sum((Affinities - TrueAvg) * (Predictions - PredAvg))
    Den = np.sqrt(np.sum((Affinities - TrueAvg)**2) * np.sum((Predictions - PredAvg)**2))
    
    return Num/Den

In [15]:
from sklearn.metrics import precision_recall_curve, auc

def compute_aupr(Affinities, Predictions):
    precision, recall, _ = precision_recall_curve(Affinities, Predictions)
    return auc(recall, precision)

In [21]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def backpropagation_two_hidden_layers(X, y, W1, b1, W2, b2, W3, b3, learning_rate):
    
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    
    Z3 = np.dot(A2, W3) + b3
    y_pred = Z3 
    
    n = y.shape[0]
    loss = np.mean((y - y_pred)**2)
    dLoss_dZ3 = (2 / n) * (y_pred - y)
    dLoss_dW3 = np.dot(A2.T, dLoss_dZ3)
    dLoss_db3 = np.sum(dLoss_dZ3, axis=0, keepdims=True)
    
    dLoss_dA2 = np.dot(dLoss_dZ3, W3.T)
    dLoss_dZ2 = dLoss_dA2 * sigmoid_derivative(Z2)
    dLoss_dW2 = np.dot(A1.T, dLoss_dZ2)
    dLoss_db2 = np.sum(dLoss_dZ2, axis=0, keepdims=True)
    
    dLoss_dA1 = np.dot(dLoss_dZ2, W2.T)
    dLoss_dZ1 = dLoss_dA1 * sigmoid_derivative(Z1)
    dLoss_dW1 = np.dot(X.T, dLoss_dZ1)
    dLoss_db1 = np.sum(dLoss_dZ1, axis=0, keepdims=True)
    
    W1 -= learning_rate * dLoss_dW1
    b1 -= learning_rate * dLoss_db1
    W2 -= learning_rate * dLoss_dW2
    b2 -= learning_rate * dLoss_db2
    W3 -= learning_rate * dLoss_dW3
    b3 -= learning_rate * dLoss_db3
    
    return W1, b1, W2, b2, W3, b3, loss

# **Data Training on Davis Dataset**

In [17]:
dd.head()

,drug_id,prot_id,SMILES,sequence amino acid,sequence amino acid affinity,encoded_sequence
0,11314340,AAK1,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,7.366532,"[13, 12, 12, 14, 14, 4, 16, 2, 2, 6, 7, 8, 8, ..."
1,11314340,ABL1(E255K),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000,"[15, 14, 18, 12, 10, 11, 3, 15, 11, 11, 6, 2, ..."
2,11314340,ABL1(F317I),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000,"[15, 14, 18, 12, 10, 11, 3, 15, 11, 11, 6, 2, ..."
3,11314340,ABL1(F317I)p,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000,"[15, 14, 18, 12, 10, 11, 3, 15, 11, 11, 6, 2, ..."
4,11314340,ABL1(F317L),CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC...,PFWKILNPLLERGTYYYFMGQQPGKVLGDQRRPSLPALHFIKGAGK...,5.000000,"[15, 14, 18, 12, 10, 11, 3, 15, 11, 11, 6, 2, ..."


In [19]:
def split_case_1(df):
    train_data = []
    test_data = []
    
    for protein, group in df.groupby('prot_id'):
        n_train = int(len(group) * 0.7)  # 70% for train
        group = group.sample(frac=1, random_state=42)  # Shuffle the data
        train_data.append(group.iloc[:n_train])
        test_data.append(group.iloc[n_train:])
    
    train = pd.concat(train_data).reset_index(drop=True)
    test = pd.concat(test_data).reset_index(drop=True)
    return train, test

def split_case_2(df):
    unique_proteins = df['prot_id'].unique()
    n_train_proteins = int(len(unique_proteins) * 0.9)  # 90% of proteins for train
    train_proteins = np.random.choice(unique_proteins, size=n_train_proteins, replace=False)
    
    train = df[df['prot_id'].isin(train_proteins)]
    test = df[~df['prot_id'].isin(train_proteins)]
    return train, test

def split_case_3(df):
    train_data = []
    test_data = []
    
    # Group by drug and split each group
    for drug, group in df.groupby('drug_id'):
        n_train = int(len(group) * 0.7)  # 70% for train
        group = group.sample(frac=1, random_state=42)  # Shuffle the data
        train_data.append(group.iloc[:n_train])
        test_data.append(group.iloc[n_train:])
    
    train = pd.concat(train_data).reset_index(drop=True)
    test = pd.concat(test_data).reset_index(drop=True)
    return train, test

def split_case_4(df):
    unique_drugs = df['drug_id'].unique()
    n_train_drugs = int(len(unique_drugs) * 0.9)  # 90% of drugs for train
    train_drugs = np.random.choice(unique_drugs, size=n_train_drugs, replace=False)
    
    train = df[df['drug_id'].isin(train_drugs)]
    test = df[~df['drug_id'].isin(train_drugs)]
    return train, test

# Generate splits
X1_train, X1_test = split_case_1(dd)  # Case 1: No new proteins in test
X2_train, X2_test = split_case_2(dd)  # Case 2: New proteins in test
X3_train, X3_test = split_case_3(dd)  # Case 3: No new drugs in test
X4_train, X4_test = split_case_4(dd)  # Case 4: New drugs in test


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

def pad_sequences(sequences, max_length=None):
    if max_length is None:
        max_length = max(len(seq) for seq in sequences)
    return [seq + [0] * (max_length - len(seq)) for seq in sequences]

def prepare_data(df):
    # Pad sequences to same length
    sequences = pad_sequences(df['encoded_sequence'].values)
    X = np.array(sequences)
    y = df['sequence amino acid affinity'].values.reshape(-1, 1)
    
    # Normalize features
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)
    
    return X, y, scaler_X, scaler_y

def forward_pass(X, W1, b1, W2, b2, W3, b3):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    
    Z3 = np.dot(A2, W3) + b3
    y_pred = Z3
    
    return y_pred, (A1, A2)

def evaluate_model(y_true, y_pred, scaler_y):
    y_true_orig = scaler_y.inverse_transform(y_true)
    y_pred_orig = scaler_y.inverse_transform(y_pred)
    
    # Calculate metrics
    mse = mean_squared_error(y_true_orig, y_pred_orig)
    pearson = Pearson_correlation(y_true_orig.flatten(), y_pred_orig.flatten())
    ci = ConcordanceIndex(y_true_orig.flatten(), y_pred_orig.flatten())
    r2 = R2(y_true_orig.flatten(), y_pred_orig.flatten())
    
    return {
        'MSE': mse,
        'Pearson': pearson,
        'CI': ci,
        'R2': r2
    }

def train_model(X_train, y_train, X_test, y_test, scaler_y, input_size=1000, hidden_size1=512, hidden_size2=256, output_size=1, 
                learning_rate=0.001, epochs=100, batch_size=32):
    # Initialize weights
    W1 = np.random.randn(input_size, hidden_size1) * np.sqrt(2. / input_size)
    b1 = np.zeros((1, hidden_size1))
    W2 = np.random.randn(hidden_size1, hidden_size2) * np.sqrt(2. / hidden_size1)
    b2 = np.zeros((1, hidden_size2))
    W3 = np.random.randn(hidden_size2, output_size) * np.sqrt(2. / hidden_size2)
    b3 = np.zeros((1, output_size))
    
    n_batches = int(np.ceil(len(X_train) / batch_size))
    
    for epoch in range(epochs):
        total_loss = 0
        
        indices = np.random.permutation(len(X_train))
        X_train = X_train[indices]
        y_train = y_train[indices]
        
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = min(start_idx + batch_size, len(X_train))
            
            batch_X = X_train[start_idx:end_idx]
            batch_y = y_train[start_idx:end_idx]
            
            y_pred, (A1, A2) = forward_pass(batch_X, W1, b1, W2, b2, W3, b3)
            
            W1, b1, W2, b2, W3, b3, batch_loss = backpropagation_two_hidden_layers(
                batch_X, batch_y, W1, b1, W2, b2, W3, b3, learning_rate
            )
            
            total_loss += batch_loss
        
        train_pred, _ = forward_pass(X_train, W1, b1, W2, b2, W3, b3)
        test_pred, _ = forward_pass(X_test, W1, b1, W2, b2, W3, b3)
        
        train_metrics = evaluate_model(y_train, train_pred, scaler_y)
        test_metrics = evaluate_model(y_test, test_pred, scaler_y)
        
        print(f"\nEpoch {epoch + 1}/{epochs}")
        print(f"Loss: {total_loss/n_batches:.4f}")
        print("Training Metrics:")
        for metric, value in train_metrics.items():
            print(f"{metric}: {value:.4f}")
        print("Test Metrics:")
        for metric, value in test_metrics.items():
            print(f"{metric}: {value:.4f}")
    
    return W1, b1, W2, b2, W3, b3

# Train for each splitting case
splitting_cases = [
    (X1_train, X1_test, "Case 1: No new proteins in test"),
    (X2_train, X2_test, "Case 2: New proteins in test"),
    (X3_train, X3_test, "Case 3: No new drugs in test"),
    (X4_train, X4_test, "Case 4: New drugs in test")
]

for train_df, test_df, case_name in splitting_cases:
    print(f"\nTraining for {case_name}")
    print("-" * 50)
    
    # Prepare data
    X_train, y_train, scaler_X, scaler_y = prepare_data(train_df)
    X_test, y_test, _, _ = prepare_data(test_df)
    
    # Train model
    train_model(X_train, y_train, X_test, y_test, scaler_y,
                input_size=X_train.shape[1],
                epochs=50,
                batch_size=32)


Training for Case 1: No new proteins in test
--------------------------------------------------

Epoch 1/50
Loss: 0.9911
Training Metrics:
MSE: 0.7647
Pearson: 0.1900
CI: 0.5810
R2: 0.0274
Test Metrics:
MSE: 0.7535
Pearson: 0.2373
CI: 0.6267
R2: 0.0416

Epoch 2/50
Loss: 0.9631
Training Metrics:
MSE: 0.7531
Pearson: 0.2171
CI: 0.5969
R2: 0.0422
Test Metrics:
MSE: 0.7384
Pearson: 0.2702
CI: 0.6416
R2: 0.0608


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, precision_recall_curve, auc
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# Data preparation (replace with actual data loading logic)
# Example input data based on the notebook structure
data = pd.DataFrame({
    "encoded_sequence": [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]],
    "affinity": [0.5, 0.8, 0.2, 0.9]
})
data = data.explode("encoded_sequence").reset_index(drop=True)
X = np.array(data["encoded_sequence"]).reshape(-1, 1, 1, 1)  # Input reshaped for CNN
Y = np.array(data["affinity"])  # Output

# Splitting cases
def split_case_1(X, Y):
    split_idx = int(len(X) * 0.7)
    return X[:split_idx], X[split_idx:], Y[:split_idx], Y[split_idx:]

# Metrics computation
def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    pcc, _ = pearsonr(y_true, y_pred)
    return mse, mae, r2, pr_auc, pcc

# AlexNet Model
def build_alexnet(input_shape):
    model = Sequential([
        Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='linear')
    ])
    return model

# ResNet Model
def build_resnet(input_shape):
    base_model = ResNet50(include_top=False, weights=None, input_shape=input_shape, pooling='avg')
    model = Sequential([
        base_model,
        Dense(1, activation='linear')
    ])
    return model

# Training Function
def train_and_evaluate_cnn(X_train, X_test, Y_train, Y_test, model_fn, input_shape, epochs=10, batch_size=32):
    model = model_fn(input_shape)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=1)

    predictions = model.predict(X_test).flatten()
    mse, mae, r2, pr_auc, pcc = compute_metrics(Y_test, predictions)
    print(f"Final Metrics:")
    print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, PR AUC: {pr_auc:.4f}, PCC: {pcc:.4f}\n")

# Example usage
X_train, X_test, Y_train, Y_test = split_case_1(X, Y)
input_shape = (1, 1, 1)  # Replace with actual input shape

print("Training AlexNet:")
train_and_evaluate_cnn(X_train, X_test, Y_train, Y_test, build_alexnet, input_shape, epochs=5)

print("Training ResNet:")
train_and_evaluate_cnn(X_train, X_test, Y_train, Y_test, build_resnet, input_shape, epochs=5)
